In [13]:
import cv2 as cv
import imutils
import numpy as np
from skimage import data, filters, feature, morphology,io, measure, exposure, draw
from matplotlib import pyplot as plt
from scipy import ndimage
from skimage.metrics import mean_squared_error

def load_templates(paths):
    templates = []
    for path in paths:
        img_rgb = cv.imread(f'templates/{path}')
        img = cv.cvtColor(img_rgb, cv.COLOR_RGB2GRAY)
        img = cv.resize(img, (30,70), interpolation = cv.INTER_AREA)
        templates.append({'val': path, 'img': img})
    return templates

In [14]:
def getSubImage(src, rect):
    box = cv.boxPoints(rect)
    box = np.int0(box)
    center, size, angle = rect

    # grab the dimensions of the image and then determine the center
    (h, w) = src.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    # grab the rotation matrix (applying the negative of the angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv.getRotationMatrix2D((cX, cY), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
       
    
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
    # perform the actual rotation and return the image
    dst = cv.warpAffine(src, M, (nW, nH))
    out = cv.getRectSubPix(dst, (int(size[0]), int(size[1])), (nW//2, nH//2))
        
    return out


class Banknote:
    def __init__(self, img_rgb, img_marked, contour):
        self.rgb = img_rgb
        self.marked = img_marked
        self.contour = contour
        self.gray = None
        self.rotated = None
        self.haveAoi = False
        self.aoi = False
        self.withAoi = None
        self.value = 'unknown'
    
    def rotate_and_resize(self):
        points = []
        for i in range(len(self.marked)):
            for j in range(len(self.marked[i])):
                if self.marked[i][j] == 255:
                    points.append([j,i])
        rect = cv.minAreaRect(np.array(points))
        self.rotated = getSubImage(self.rgb, rect)
        if self.rotated.shape[1] < self.rotated.shape[0]:
            self.rotated  = ndimage.rotate(self.rotated, -90)
        self.rotated = imutils.resize(self.rotated, height=200, width=400)
    
    def find_aoi(self):
        self.gray = cv.cvtColor(self.rotated, cv.COLOR_RGB2GRAY)
        banknote = cv.GaussianBlur(self.gray, (5, 5), 0)
        banknote = exposure.adjust_gamma(banknote, gamma=0.7)
        banknote = cv.Canny(banknote,100,100) 
        for i in range(2):
            banknote = morphology.dilation(banknote)
   
        contours = measure.find_contours(banknote)
        for contour in contours:
            coords = measure.approximate_polygon(contour, tolerance=2.5)
            rr, cc = draw.polygon(coords[:, 1], coords[:, 0], shape=None)
            if rr.size >0 and cc.size >0 :
                (x,y,w,h) = cv.boundingRect(np.array([[min(rr),min(cc)],[max(rr),max(cc)]]))
                if w < 60 and w > 40 and h > 60 and h <100 and x < 40 and y < 40:
                    self.withAoi = cv.rectangle(self.rotated, (x,y), (x+w, y+h), (255,255,0), 2)
                    self.aoi = self.gray[y:y+h, x:x+w]
                    self.aoi = self.better_aoi(self.aoi)
                    self.aoi = cv.resize(self.aoi, (30,70), interpolation = cv.INTER_AREA)
                    self.haveAoi = True
        if self.haveAoi == False:
            self.withAoi = self.rotated

    def better_aoi(self, banknote):
        banknote = cv.Canny(banknote,100,100)
        banknote = morphology.dilation(banknote)
        return banknote
    
    def find_value(self):
        if self.haveAoi:
            matching = [0 for _ in templates]
            for idx,template in enumerate(templates):
                matching[idx] = mean_squared_error(self.aoi, template['img'])
            self.value = int(templates[np.argmin(matching)]['val'].split('.')[0])
            
    def check_again(self):
        if self.value == 'unknown':
            self.rotated = ndimage.rotate(self.rotated, 180)
            self.find_aoi()
            self.find_value()

        
    
    

In [15]:
class Image:
    def __init__(self, etap, path):
        self.rgb = None
        self.gray = None
        self.afterNoiseReduction = None
        self.withMarkedBanknotes = None
        self.banknotes = []
        self.overallValue = 0
        self.withValues = None
        
        self.load(etap, path)
    
    def load(self, etap, path):
        self.rgb = cv.cvtColor(cv.imread(f'{etap}/{path}'), cv.COLOR_BGR2RGB)
        self.gray  = cv.cvtColor(self.rgb, cv.COLOR_RGB2GRAY)
        
    def noise_reduction(self):
        image = self.gray
        image = cv.GaussianBlur(image, (5, 5), 0)
        _, image = cv.threshold(image,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
        kernel = np.ones((5,5), np.uint8)
        image = cv.erode(image, kernel, iterations=3)
        image = cv.dilate(image, kernel, iterations=3)
        self.afterNoiseReduction = image
         
    def mark_banknotes(self):
        image = self.afterNoiseReduction
        contours = measure.find_contours(image, fully_connected='high', positive_orientation='high')
        polygons = np.zeros(image.shape)
        for contour in contours:
            coords = measure.approximate_polygon(contour, tolerance=2.5)
            rr, cc = draw.polygon(coords[:, 1], coords[:, 0], shape=None)
            polygons[cc, rr] = 255

        self.withMarkedBanknotes = polygons
    
    def find_banknotes(self):
        contours = measure.find_contours(self.withMarkedBanknotes)
        for contour in contours:
            coords = measure.approximate_polygon(contour, tolerance=2.5)
            rr, cc = draw.polygon(coords[:, 1], coords[:, 0], shape=None)
            if rr.size >0 and cc.size >0:
                (x,y,w,h) = cv.boundingRect(np.array([[min(rr),min(cc)],[max(rr),max(cc)]]))        

                if (w > 200 and h > 100) or (w >100 and h >200):
                    one_banknote_on_img_rgb = np.full(self.rgb.shape, [np.uint8(0),np.uint8(0),np.uint8(0)])
                    one_banknote_on_img_marked = np.full(self.withMarkedBanknotes.shape, np.uint8(0))
                    for y_idx,x_idx in zip(rr,cc):
                        one_banknote_on_img_rgb[x_idx,y_idx] = self.rgb[x_idx,y_idx]
                        one_banknote_on_img_marked[x_idx,y_idx] = self.withMarkedBanknotes[x_idx,y_idx]


                    banknote_rgb = one_banknote_on_img_rgb[y:y+h,x:x+w]        
                    banknote_marked = one_banknote_on_img_marked[y:y+h,x:x+w]

                    self.banknotes.append(Banknote(banknote_rgb, banknote_marked, contour))
                
    def show_banknotes(self, option, title):
        l = len(self.banknotes)
        if l >0:
            fig, axs = plt.subplots(l, 1, figsize=(10, 10))
            fig.suptitle(title)
            if l > 1:
                for i, banknote in enumerate(self.banknotes):
                    if getattr(banknote, option) is not None: 
                        axs[i].imshow(getattr(banknote, option))
            else:
                if getattr(self.banknotes[0],option) is not None:
                    axs.imshow(getattr(self.banknotes[0],option))
    
    
    def process_banknotes(self):
        for banknote in self.banknotes:
            banknote.rotate_and_resize()
            
        self.show_banknotes('rotated','Obrócone i odpowiednio dopasowane rozmiarem banknoty')
        for banknote in self.banknotes:
            banknote.find_aoi()
        self.show_banknotes('withAoi','Banknoty z zaznaczonymi wartościami 1')
        for banknote in self.banknotes:
            banknote.find_value()
            banknote.check_again()
        self.show_banknotes('withAoi','Banknoty z zaznaczonymi wartościami 2')
    
    def sumValues(self):
        self.overallValue = sum([banknote.value if banknote.value != 'unknown' else 0 for banknote in self.banknotes ])
   
    
    def draw_contours(self):
        for banknote in self.banknotes:
            if banknote.value == 'unknown':
                 print("Nie rozpoznano")
            else:
                print("Banknot o wartosci: ", banknote.value, "zł")
                pts = np.array(list(zip(banknote.contour[:,1], banknote.contour[:,0])), np.int32)
                self.withValues = cv.polylines(self.rgb, [pts],True,(255,0,0),thickness = 10)    
                font = cv.FONT_HERSHEY_SIMPLEX
                center = np.array(np.mean(banknote.contour, axis=0),np.int32)
                self.withValues = cv.putText(self.withValues, str(banknote.value),
                                             [center[1]-20,center[0]], font, 2,(255,0,0),10,cv.LINE_AA)
    


In [23]:
def process_image(etap, path):
    fig, axs = plt.subplots(3,1, figsize=(10,15))
    image = Image(etap, path)
    axs[0].imshow(image.rgb)

    image.noise_reduction()
    axs[1].imshow(image.afterNoiseReduction,cmap='gray')
    
    image.mark_banknotes()
    axs[2].imshow(image.withMarkedBanknotes,cmap='gray')
    
    image.find_banknotes()
    image.show_banknotes('rgb', 'Znalezione banknoty')
    
    image.process_banknotes()
    image.sumValues()
    image.draw_contours()
    
    fig, axs = plt.subplots(1,1, figsize=(10,15))
    if image.overallValue != 0:
        axs.imshow(image.withValues)
    else:
        axs.imshow(image.rgb)

    
    print("Wartość gotówki na zdjęciu: ", image.overallValue, "zł")
    print("===========================")

In [24]:
images_paths = ['0.jpg','1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg']
templates = load_templates(['10.jpg','20.jpg','50.jpg','100.jpg'])


In [ ]:
process_image('etap_3', images_paths[6])

Nie rozpoznano
Banknot o wartosci:  10 zł
Banknot o wartosci:  100 zł
Wartość gotówki na zdjęciu:  110 zł
